In [ ]:
## Retail Referrals program

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())
spark

In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/sharb24/SBB/thural2/record.py').read())


In [ ]:
###################################################################################################
############################# RETAIL REFERRALS ####################################################

## this pulls all referrals on a weekly basis
dfret=spark.sql("select * from anp_cabbtdct1_final.SBB_SC_RETAIL_REFERRALS") 
df_ret=dfret.toPandas()

df_ret1=df_ret[(df_ret.record_date>=fy_start) & (df_ret.record_date<=rec_date)]

## add a new record_date variable
retailref=df_ret1.copy()

## add the month
retailref['month_dt'] = pd.to_datetime(retailref.closed_dt, format='%Y-%m-%d')
retailref['month_dt'] = retailref['month_dt'].dt.month

import calendar
retailref['month_dt'] = retailref['month_dt'].apply(lambda x: calendar.month_name[x]).str.lower()
print(retailref.month_dt.unique())


## add the quarter
tuple_q1=('november','december','january')
tuple_q2=('february','march','april')
tuple_q3=('may','june','july')
tuple_q4=('august','september','october')

def my_callback(row): 
    if row['month_dt'] in tuple_q1: 
        return 'Q1' 
    elif row['month_dt'] in tuple_q2: 
        return 'Q2' 
    elif row['month_dt'] in tuple_q3: 
        return 'Q3' 
    elif row['month_dt'] in tuple_q4: 
        return 'Q4'

retailref['quarter']=retailref.apply(my_callback, axis=1)


In [ ]:
######## BUSINESS LOGIC: AMSB only gets credit once for one customer per quarter##################

#####remove duplicates - AMSB gets credit once for same customer
## order the leads by acf2,cust_no,and earliest closed_dt and then get the first record
retailref = retailref.sort_values(['orig_acf2','final_id','closed_dt','quarter'], ascending=[True, True,True,True])
retail2 = retailref.groupby(['orig_acf2','final_id','quarter'], as_index=False).head(1)

## ensure the order of vars and save into a table
retail2=retail2[['lead_event_id','cust_id','orig_acf2','job_clasfcn_cd','cc_intrnl_org_prty_id',
'origtr_branch_no','origtr_line_of_busnes_ds','asignd_to_branch_no','receive_acf2',
'new_receiving_lob','initve_prodct_target_ds','initve_prodct_target_cd',
'lifecy_reason_ds','lifecy_reason_cd','creatn_dt','action_dt', 'closed_dt',
'lead_source_cd','cust_no','final_id','month_dt','quarter','record_date']]


In [ ]:
### WRITE THE DATA TO A UNIQUE CUST TABLE
#retail2.write.mode("overwrite").format("hive").saveAsTable("anp_cabbtdct1_working.tbl_bb4179_final_20210813")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(retail2).write.mode("overwrite").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_RETAIL_REF_CUST")




In [ ]:
### query the unique cust table
dfret0=spark.table('anp_cabbtdct1_final.SBB_SC_RETAIL_REF_CUST') 
retail3=dfret0.toPandas()

## filter
retail3=retail3[(retail3.record_date>=fy_start)]

### get hr/alignment
df=spark.sql("select am_cost_center,am_cost_center_name,am_cost_center_full_name, \
                     sm_cost_center,sm_cost_center_name from \
                     (select *, row_number() over(partition by am_cost_center order by effective_mon desc) as ran \
                     from anp_cabbtdct1_final.HRM_SBB_ALIGNMENT0)b where ran=1")
dfhr=df.toPandas()

dfhr=dfhr[['am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']].drop_duplicates()

retail35=pd.merge(retail3,dfhr,left_on=['cc_intrnl_org_prty_id'],right_on=['am_cost_center'],
                   how="left")

## include the metric name and scorecard date
retail35['metric_name']='AMSB Retail Referrals'


In [ ]:
#### count referrals
retail34=retail35.groupby(by=['orig_acf2','month_dt','metric_name'],as_index=False).aggregate({
                    'final_id': 'count'
                     })

## transpose the counts
retail36=retail34.pivot_table(index=['orig_acf2','metric_name'], 
                    columns=['month_dt'],
                     values='final_id', aggfunc='first', fill_value=0).reset_index()


In [ ]:
#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

################## CHANGE EVERY FY #############################

complementary = [c for c in net_a if c not in retail36]
retail37= pd.concat([net_a[complementary], retail36], axis=1, join='outer')

##delete the extra filler row
retail37= retail37.drop(labels='a', axis=0)
##convert to float
retail37=retail37.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

### merge back to get final AMSB vars
    ### get hr/alignment
df1=spark.sql("select acf2_id,employee_name,am_cost_center,am_cost_center_name,am_cost_center_full_name, \
                     sm_cost_center,sm_cost_center_name from \
                     (select *, row_number() over(partition by acf2_id order by record_date desc) as ran \
                     from anp_cabbtdct1_final.HRM_SBB_ALIGNMENT0)b where ran=1")
dfhr=df1.toPandas()

retail38=pd.merge(dfhr,retail37,left_on=['acf2_id'],right_on=['orig_acf2'],
                   how="inner")

retail38['level']='AMSB'
retail38['scorecard_filter']='SBB'
retail38['record_date']=rec_date


In [ ]:
##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(retail38)

## replace 0 with NaN so median only counts non-zeros
retail38=retail38.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_amsb_medianq1=retail38.groupby(['metric_name','sm_cost_center'], as_index=False)['q1'].median()
df_amsb_medianq2=retail38.groupby(['metric_name','sm_cost_center'], as_index=False)['q2'].median()
df_amsb_medianq3=retail38.groupby(['metric_name','sm_cost_center'], as_index=False)['q3'].median()
df_amsb_medianq4=retail38.groupby(['metric_name','sm_cost_center'], as_index=False)['q4'].median()
df_amsb_medianytd=retail38.groupby(['metric_name','sm_cost_center'], as_index=False)['ytd'].median()
##rename median
df_amsb_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_amsb_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_amsb_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_amsb_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_amsb_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
ret_pt11=pd.merge(pd.merge(retail38,df_amsb_medianq1,
                on=['metric_name','sm_cost_center'],
                how='left'),df_amsb_medianq2,on=['metric_name','sm_cost_center'],how='left')

ret_pt22=pd.merge(pd.merge(ret_pt11,df_amsb_medianq3,
                on=['metric_name','sm_cost_center'],
                how='left'),df_amsb_medianq4,on=['metric_name','sm_cost_center'],how='left')

ret_pt338=pd.merge(ret_pt22,df_amsb_medianytd,on=['metric_name','sm_cost_center'],how="left")


### get AMSBs
amsbret=ret_pt338[['metric_name','level','acf2_id','employee_name','november','december','january',
        'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name',
           'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
           'scorecard_filter','record_date']]


In [ ]:
###################################################################################################
#################################### ROLL UP TO AM (SBB FILTER) ################################################

##group AMSB results
amret2=amgrpfunc(amsbret)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(amret2)

## replace 0 with NaN so median only counts non-zeros
amret2=amret2.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_am_medianq1=amret2.groupby(['metric_name'], as_index=False)['q1'].median()
df_am_medianq2=amret2.groupby(['metric_name'], as_index=False)['q2'].median()
df_am_medianq3=amret2.groupby(['metric_name'], as_index=False)['q3'].median()
df_am_medianq4=amret2.groupby(['metric_name'], as_index=False)['q4'].median()
df_am_medianytd=amret2.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_am_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_am_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_am_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_am_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_am_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
ret1=pd.merge(pd.merge(amret2,df_am_medianq1,
                on=['metric_name'],
                how='left'),df_am_medianq2,on=['metric_name'],how='left')

ret2=pd.merge(pd.merge(ret1,df_am_medianq3,
                on=['metric_name'],
                how='left'),df_am_medianq4,on=['metric_name'],how='left')

ret3=pd.merge(ret2,df_am_medianytd,on=['metric_name'],how="left")

ret3['level']='AM'
ret3['scorecard_filter']='SBB'
ret3['employee_name']=ret3.am_cost_center_full_name
ret3['acf2_id']=''

### get AM
retail_am=ret3[['metric_name','level','acf2_id','employee_name','november','december','january',
        'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center',
            'sm_cost_center_name','scorecard_filter','record_date']]

## create a copy for Total AM
retail_am1=retail_am.copy()
retail_am1['scorecard_filter']='Total'


In [ ]:
###################################################################################################
#################################### ROLL UP TO SM ################################################

sm_ret=smgrpfunc(retail_am)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(sm_ret)

## replace 0 with NaN so median only counts non-zeros
sm_ret=sm_ret.replace(0.0, np.nan)


### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_sm_medianq1=sm_ret.groupby(['metric_name'], as_index=False)['q1'].median()
df_sm_medianq2=sm_ret.groupby(['metric_name'], as_index=False)['q2'].median()
df_sm_medianq3=sm_ret.groupby(['metric_name'], as_index=False)['q3'].median()
df_sm_medianq4=sm_ret.groupby(['metric_name'], as_index=False)['q4'].median()
df_sm_medianytd=sm_ret.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_sm_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_sm_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_sm_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_sm_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_sm_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
sm_ret1=pd.merge(pd.merge(sm_ret,df_sm_medianq1,
                on=['metric_name'],
                how='left'),df_sm_medianq2,on=['metric_name'],how='left')

sm_ret2=pd.merge(pd.merge(sm_ret1,df_sm_medianq3,
                on=['metric_name'],
                how='left'),df_sm_medianq4,on=['metric_name'],how='left')

sm_ret3=pd.merge(sm_ret2,df_sm_medianytd,on=['metric_name'],how="left")

sm_ret3['acf2_id']=''
sm_ret3['level']='SM'
sm_ret3['scorecard_filter']='SBB'
sm_ret3['employee_name']=sm_ret3.sm_cost_center_name
sm_ret3['am_cost_center']=''
sm_ret3['am_cost_center_name']=''
sm_ret3['am_cost_center_full_name']=''

sm_retail=sm_ret3[['metric_name','level','acf2_id','employee_name','november','december','january',
        'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center',
            'sm_cost_center_name','scorecard_filter','record_date']]

## make a copy for Total
sm_retail1=sm_retail.copy()
sm_retail1['scorecard_filter']='Total'


In [ ]:
#################################### ROLL UP TO NATIONAL OFFICE ##################################

nso_ret=natgrpfunc(sm_retail)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(nso_ret)

## replace 0 with NaN so median only counts non-zeros
nso_ret=nso_ret.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_nso_medianq1=nso_ret.groupby(['metric_name'], as_index=False)['q1'].median()
df_nso_medianq2=nso_ret.groupby(['metric_name'], as_index=False)['q2'].median()
df_nso_medianq3=nso_ret.groupby(['metric_name'], as_index=False)['q3'].median()
df_nso_medianq4=nso_ret.groupby(['metric_name'], as_index=False)['q4'].median()
df_nso_medianytd=nso_ret.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_nso_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_nso_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_nso_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_nso_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_nso_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
nso_ret1=pd.merge(pd.merge(nso_ret,df_nso_medianq1,
                on=['metric_name'],
                how='left'),df_nso_medianq2,on=['metric_name'],how='left')

nso_ret2=pd.merge(pd.merge(nso_ret1,df_nso_medianq3,
                on=['metric_name'],
                how='left'),df_nso_medianq4,on=['metric_name'],how='left')

nso_ret3=pd.merge(nso_ret2,df_nso_medianytd,on=['metric_name'],how="left")

nso_ret3['acf2_id']=''
nso_ret3['level']='National Office'
nso_ret3['scorecard_filter']='Total'
nso_ret3['employee_name']='National Office'
nso_ret3['am_cost_center']=''
nso_ret3['am_cost_center_name']=''
nso_ret3['am_cost_center_full_name']=''
nso_ret3['sm_cost_center']=''
nso_ret3['sm_cost_center_name']=''

nso_ret3['record_date']=rec_date

nso_retail=nso_ret3[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center',
            'sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
############################## CONCATENATE ALL DATAFRAMES ########################################
## amsb,am sbb, am total, sm sbb, sm total
retail_ref_all=pd.concat([amsbret,retail_am,retail_am1,sm_retail,sm_retail1,nso_retail],axis=0)


In [ ]:
## fill in with zeroes
retail_ref_allf=retail_ref_all.replace(np.nan,0.0)

In [ ]:
retail_ref_allf.record_date.unique()

In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(retail_ref_allf).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_OUTPUT_RETAIL_REF") 

In [ ]:
### END OF PROGRAM
nso_retail[(nso_retail.metric_name=="AMSB Retail Referrals")]#&(nso_retail.employee_name=="GTA")]